In [124]:
import torch
import sys
import os
from pathlib import Path
import numpy as np
from typing import List
from PIL import Image
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

sys.path.append(".")
sys.path.append("..")
sys.path.append(os.path.abspath('D:/Docs/Kursach/stylegan3-editing/utils/'))

from models.stylegan3.model import SG3Generator
from common import make_transform, tensor2im
from editor.Editor import FaceEditor
from models.stylegan3.model import GeneratorType

In [125]:
generator = SG3Generator(checkpoint_path="../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl").decoder

num_images_per_cluster = 5
trunc = 0.3

INTERFACEGAN_RANGES = {
    "age": (-5, 5),
    "smile": (-5, 5),
    "Male": (-5, 5),
    'glasses': (-5, 5),
}

Loading StyleGAN3 generator from path: ../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl
Done!


In [126]:
csv_name = 'iris'
df = pd.read_csv(f"{Path('./datasets/') / csv_name}.csv")
df = df.select_dtypes(exclude=['object'])

In [127]:
df = df.to_numpy()
df

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [128]:
def pca_boundary_adjustment(df, num_components):
    pca = PCA(n_components=num_components)
    pca_components = pca.fit_transform(df)
    return pca_components

In [129]:
scaler = MinMaxScaler()
#df = pca_boundary_adjustment(df,4)
#df_scaled = scaler.fit_transform(df)
df_scaled = df

In [130]:
editor = FaceEditor(generator, GeneratorType.ALIGNED)
start_path = Path(f'./edit_results/{csv_name}/boundary_method_without_scaler_w')
start_path.mkdir(exist_ok=True, parents=True)

In [131]:
def save_coupled_images(images: List, output_path: Path):
    if type(images[0]) == list:
        images = [image[0] for image in images]
    res = np.array(images[0])
    for image in images[1:]:
        res = np.concatenate([res, image], axis=1)
    res = Image.fromarray(res).convert("RGB")
    res.save(output_path)

In [ ]:
z = torch.from_numpy(np.random.randn(1, 512)).to('cuda')
w = generator.mapping(z, None, truncation_psi=trunc)
for img,i in zip(df_scaled,range(len(df_scaled))):
    edit_image, _ = editor.edit_more_then_one(latents=w,
                                            directions = ["age", "smile", "Male", 'glasses'],
                                            factor_ranges=INTERFACEGAN_RANGES,
                                            apply_user_transformations=False,
                                            direction_strength = img)
    save_coupled_images(edit_image, output_path=start_path / f"{i}.png")